In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2,4))

net(X)

tensor([[-0.0347],
        [ 0.0581]], grad_fn=<AddmmBackward>)

In [2]:
print(net[0])

Linear(in_features=4, out_features=8, bias=True)


In [3]:
print(net[2])

Linear(in_features=8, out_features=1, bias=True)


In [4]:
print(net[2].state_dict()) # 通过这个state_dict取出来

OrderedDict([('weight', tensor([[-0.1934,  0.0551,  0.3258, -0.2951,  0.1112, -0.0013,  0.1831,  0.1510]])), ('bias', tensor([0.0232]))])


In [5]:
print(type(net[2].bias))

<class 'torch.nn.parameter.Parameter'>


In [6]:
print(net[2].bias)

Parameter containing:
tensor([0.0232], requires_grad=True)


In [7]:
print(net[2].bias.data)

tensor([0.0232])


In [8]:
net[2].weight.grad == None  # 这里还没反向计算

True

In [9]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))


In [13]:
print(*[(name, param.shape) for name, param in net.named_parameters()])

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [14]:
net.state_dict()['2.bias'].data # 知道名字后就可以这样取得

tensor([0.0232])

In [17]:
def block1():
    return nn.Sequential(nn.Linear(4,8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

In [20]:
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f"block{i+2}", block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4,1))
rgnet(X)

tensor([[0.6542],
        [0.6550]], grad_fn=<AddmmBackward>)

In [21]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block4): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block5): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


# 内置初始化


In [22]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01) # 替换
        nn.init.zeros_(m.bias) # 直接置零

net.apply(init_normal) # 遍历一遍做修改
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0018, -0.0099,  0.0060, -0.0092]), tensor(0.))

In [23]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.constant(m.weight, 1) # 替换
        nn.init.zeros_(m.bias) # 直接置零

net.apply(init_normal) # 遍历一遍做修改
net[0].weight.data[0], net[0].bias.data[0]

/Users/xujian/miniconda3/envs/dev_torch3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  This is separate from the ipykernel package so we can avoid doing imports until


(tensor([1., 1., 1., 1.]), tensor(0.))

In [28]:
def xavier(m): # 前面讲过了
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight) 
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42) # 42的梗。。。银河系漫游指南|
        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.1064,  0.5318,  0.2190, -0.0157])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [30]:
def my_init(m):
    if type(m) == nn.Linear:
        print("init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init) 
net[0].weight[:2]

init weight torch.Size([8, 4])
init weight torch.Size([1, 8])


tensor([[0.0000, 0.0000, 8.9729, -0.0000],
        [7.7907, -0.0000, 5.5660, -0.0000]], grad_fn=<SliceBackward>)

In [38]:
# 更暴力的方法
net[0].weight.data[:] += 1
net[0].weight.data[0,0] = 42

In [39]:
net[0].weight.data[0]

tensor([42.0000,  4.0000, 12.9729,  4.0000])

# 参数共享

In [40]:
# 参数绑定
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4,8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8,1))
net(X)

tensor([[0.2267],
        [0.2269]], grad_fn=<AddmmBackward>)

In [41]:
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
